In [1]:
# Imports for gplearn and pydotplus in order to see graph view

In [2]:
#tree viz
%pylab inline
from IPython.display import Image
import pydotplus
#gplearn
from gplearn.genetic import SymbolicRegressor
from gplearn.fitness import make_fitness

#--Import the standard libraries--
import math
import random
import matplotlib.pyplot as plt
import numpy as np

#imports for bokeh
from ipywidgets import interact

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Range1d, LinearAxis

output_notebook()
 
#prints more verbose output
DEBUG = True
verbose_value = 0
if(DEBUG): verbose_value = 1

Populating the interactive namespace from numpy and matplotlib


Loading BokehJS ...


# Read in test data

## format:

### details:

|#step  |#stride |
|-------|--------|

### training data:

|#rows  |1|
|-------|-|
|row1   | |
|row2   | |
|...    | |
|lastrow| |

Note, training data is also used as the test data in this case

In [3]:
#specfy folder/filename for data to be read in
folder = "data" 
filename = "testsine.mat"
filename_full = folder + "\\" +filename

#gets the step, str stride, hidden
def get_steps_stride():
    with open(filename_full, 'r') as file:
        line = file.readline()
        data_temp = list(map(int,line[::2]))
        #strip off the # of hidden nodes since we don't need
        data = tuple(data_temp[:2])
    return data

#returns a tuple of number of rows fortraining data
def get_num_rows():
    with open(filename_full, 'r') as file:
            lines = file.readlines()
            #Read in the num of rows for training data
            line = lines[1]
            num_rows = int(line.split(" ")[0])
    return num_rows

data_raw = np.genfromtxt(filename_full, dtype = "float64", skip_header = 2)
#get num rows is not used because it was incorrect for some of the datasets
num_rows = data_raw.shape[0]
steps,stride = get_steps_stride()

#calculate the max number of samples given the steps and stride
samples = (num_rows)-(steps+1)*stride


#If debug mode, print more verbose details
if(DEBUG):print("num_rows:{} steps:{} strides: {} samples: {}".format(num_rows,steps,stride,samples))

num_rows:101 steps:2 strides: 1 samples: 98


# Plot the data in bokeh, x dimension is unknown.

In [4]:
x_max = 1.0
x = np.linspace(0, x_max, num_rows)
y = data_raw


In [5]:

plot = figure(title=filename+" input",
              plot_height=300,
              plot_width=600,
              y_range=(min(data_raw)-1,max(data_raw)+1),
              x_range=(0,1),
              background_fill_color='#efefef',
              toolbar_location=None,
              active_drag=None)
line = plot.line(x, y, color="#8888cc", line_width=1.5, alpha=0.8)

In [6]:
show(plot)

# Setup the training data



In [7]:
stack_input = [(data_raw[begin:begin+(steps+1)*stride:stride])[0:steps] for begin in range(0,samples)]
stack_target = [(data_raw[begin:begin+(steps+1)*stride:stride])[-1] for begin in range(0,samples)]
num_samples = samples
train_x = np.stack(stack_input)
train_y = np.stack(stack_target)


# Custom Fitness function
Here is where the real usage of gplearn begins, here we define a fitness function that tests the diffrence from the result to the selected training values using RMSE. This function is actually built into gplearn, but figured it was better to show how to make your own fitness function

In [8]:
#fittness function

def _custom_fitness(y, y_pred, w):
    return np.sqrt(np.mean((y_pred-y)**2))

custom_fitness = make_fitness(_custom_fitness, greater_is_better=False)

# Symbolic regression

We run a symbolic regression on a 1000 individuals for 20 generations. There are three kinds of mutation, subtree, hoist, and point. The hoist mutation is the only type that differs significantly from what was talked about in class. In it, a random subtree is selected, then, a random subtree from within it is lifted to it’s root node. It is used to combat tree bloat, and it appears to be very effective. Without the hoist mutation, tree trees grew like redwoods.

In [9]:
#This part sets up the symbolic regressor
est = SymbolicRegressor(population_size=1000,
                           generations=20, stopping_criteria=0,
                           p_crossover=0.7, p_subtree_mutation=0.1,
                           p_hoist_mutation=0.05, p_point_mutation=0.1,
                           max_samples=0.9, verbose=verbose_value,
                           parsimony_coefficient=0.001, random_state=0,
                           metric = custom_fitness,
                           function_set=('add', 'sub', 'mul', 'div',"sin")
                           )
#This part runs it on our data
est.fit(train_x, train_y)

    |    Population Average   |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    26.67 192.94726941105804        5 0.140173091053324 0.140173091053324      7.29s
   1     9.16 3.1711157857200187        7 0.05373892328925863 0.05373892328925863     16.70s
   2     4.85 0.8980942751097309        5 0.028475677347945306 0.028475677347945306     14.95s
   3     3.74 145.61676924085847        5 0.028475677347945306 0.028475677347945306     12.81s
   4     6.77 1.017970489912353        5 0.028475677347945306 0.028475677347945306     12.67s
   5      5.3 1.7384375410430435        5 0.028475677347945306 0.028475677347945306     12.34s
   6     5.14 0.5668066974817846        5 0.028475677347945306 0.028475677347945306     11.78s
   7      5.3 0.8974308414365505        5 0.028475677347945306 0.028475677347945306     11.09s
   8

SymbolicRegressor(const_range=(-1.0, 1.0),
         function_set=('add', 'sub', 'mul', 'div', 'sin'), generations=20,
         init_depth=(2, 6), init_method='half and half', max_samples=0.9,
         metric=<gplearn.fitness._Fitness object at 0x000001E7FCF90828>,
         n_jobs=1, p_crossover=0.7, p_hoist_mutation=0.05,
         p_point_mutation=0.1, p_point_replace=0.05,
         p_subtree_mutation=0.1, parsimony_coefficient=0.001,
         population_size=1000, random_state=0, stopping_criteria=0,
         tournament_size=20, verbose=1, warm_start=False)

# Plot The function it predicted in comparison with the actual function

In [10]:
predict_y = est.predict(train_x)
x = np.arange(0, samples)

In [62]:
#testomg data sources
data = {'x': [1]}
source_1 = ColumnDataSource(data=data)

In [63]:
##build plot 1

plot_1 = figure(title="output",
              plot_height=300,
              plot_width=600,
              y_range=(min(data_raw)-1,max(data_raw)+1),
              x_range=(0,samples),
              background_fill_color='#efefef',
              toolbar_location=None,
              active_drag=None)


plot_1.grid.grid_line_color = "#9b9b9b"
plot_1.xaxis.ticker = [x for x in range (0,samples,5)]
plot_1.xaxis.axis_label = "Samples"

line_1 = plot_1.line(x, train_y, color="#8888cc", line_width=1.5, alpha=0.8)
line_2 = plot_1.line(x, predict_y, color="#9e0b0b", line_width=1.5, alpha=0.8)


h_line = plot_1.ray(x="x", y=min(data_raw)-1, length=40, angle=90,
      angle_units="deg", color="#FB8072", line_width=2, source = source_1)




In [64]:
y_upper = 10
x_upper = 10
plot_2 = figure(title="output",
              plot_height=300,
              plot_width=300,
              y_range=(0,y_upper),
              x_range=(0,x_upper),
              background_fill_color='#efefef',
              toolbar_location=None,
              active_drag=None)

#source = ColumnDataSource(dict(x=range(0,100),y=range(0,100)))

In [69]:
def update_hline(new_x):
    new_array = np.asarray([new_x])
    #for value in h_line.data_source.data:
        #print(value)
    h_line.data_source.data['x'] = new_array
    push_notebook(handle = h1)
    
def update_lim(x_lim,y_lim):
    plot_2.y_range.start = 0 
    plot_2.y_range.end = y_lim 
    plot_2.x_range.start = 0 
    plot_2.x_range.end = x_lim 
    #line_2.data_source.data['y']
    push_notebook(handle = h2)

In [70]:
h1 = show(plot_1, notebook_handle=True) 
interact(update_hline,new_x = (0,samples))
h2 = show(plot_2, notebook_handle=True) 

    


interactive(children=(IntSlider(value=49, description='new_x', max=98), Output()), _dom_classes=('widget-inter…

In [26]:
interact(update_lim,x_lim = (0,10), y_lim = (0,10))

interactive(children=(IntSlider(value=5, description='x_lim', max=10), IntSlider(value=5, description='y_lim',…

<function __main__.update_lim(x_lim, y_lim)>

In [16]:
def update():
    line_2.data_source.data['y']
    

In [17]:
print(best_gp._program)


NameError: name 'best_gp' is not defined

In [ ]:
graph = pydotplus.graphviz.graph_from_dot_data(best_gp._program.export_graphviz())
Image(graph.create_png())